In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving erd_tweets_llm_annotation.xlsx to erd_tweets_llm_annotation.xlsx


In [ ]:
df= pd.read_excel('erd_tweets_llm_annotation.xlsx')
df.head(2)

,Unnamed: 0,tweet_text,cg_4_dashboard_chainofthought_few_shot,tweet_id,tweet_text.1,tweet_date,tweet_location,user_id,screen_name,name,gender,age_group,user_location,province_code
0,1,#أردوغان: الجيشان #التركي و #السوري الحر سيواص...,3,1087010909866938368,#أردوغان: الجيشان #التركي و #السوري الحر سيواص...,2019-01-20 15:36:11,NaN,826790275666280449,NaN,NaN,NaN,NaN,NaN,NaN
1,2,@lolostars2 أردوغان حارق العبيد,2,1029535238517661696,@lolostars2 أردوغان حارق العبيد,2018-08-15 01:08:23,NaN,1474119686,MohammedtalatF,محمد طلعت فرج,male,>=40,Kingdom of Saudi Arabia,61.0


In [ ]:
df['cg_4_dashboard_chainofthought_few_shot'].value_counts()

cg_4_dashboard_chainofthought_few_shot
1    5784
3    3626
2    2015
Name: count, dtype: int64

Topic Modeling-LDA

In [ ]:
!pip install qalsadi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.3/264.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.6/332.6 kB 36.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 31.4 MB/s eta 0:00:00
  Created wheel for pickledb: filename=pickleDB-0.9.2-py3-none-any.whl size=4254 sha256=2a55bf3320e074868a3fce7c99da3ee90cf0a8f5d081381014585e1552b180f9
  Stored in directory: /root/.

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import re
import nltk

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')  # Ensure that the punkt tokenizer is downloaded

# Define a preprocessing function
def preprocess(text):
    stop_words = set(stopwords.words('arabic'))
    additional_stopwords = set(["من", "في", "على", ".", "انا", "والله", "يارب", "أردوغان", "اردوغان", "الطيب", "طيب", "تركيا", "التركية", "التركي"])  # Add other stopwords as needed
    stop_words = stop_words.union(set(string.punctuation), additional_stopwords)

    # Clean and preprocess the text
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuations
    text = re.sub(r'\d+', '', text)  # Remove numbers
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

# Assuming you have a DataFrame 'df' already loaded with relevant data
# df = pd.read_csv('your_file.csv')  # Load your data here

# Process and analyze each class separately
for class_label in [1, 2, 3]:
    # Filter df for the current class
    df_class = df[df['cg_4_dashboard_chainofthought_few_shot'] == class_label]

    # Preprocess text data
    df_class['processed_text'] = df_class['tweet_text'].apply(preprocess)

    # Vectorize text data
    vectorizer = CountVectorizer(max_df=0.9, min_df=2, token_pattern='\w+|\$[\d\.]+|\S+')
    data_vectorized = vectorizer.fit_transform(df_class['processed_text'])

    # Fit LDA model
    lda_model = LatentDirichletAllocation(n_components=5, random_state=0)
    lda_model.fit(data_vectorized)

    # Print top words for each topic
    def print_top_words(model, feature_names, n_top_words):
        for topic_idx, topic in enumerate(model.components_):
            message = f"Topic #{topic_idx}: "
            message += " ".join([feature_names[i]
                                 for i in topic.argsort()[:-n_top_words - 1:-1]])
            print(message)

    print(f"Top words for class {class_label}:")
    print_top_words(lda_model, vectorizer.get_feature_names_out(), 10)
    print("\n")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-10-5e79f8f51454>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_class['processed_text'] = df_class['tweet_text'].apply(preprocess)


Top words for class 1:
Topic #0: حزب الرئيس الانتخابات rt العدالة ان والتنمية الشعب أوغلو الانتخابات_التركية
Topic #1: الرئيس rt رئيس رجب اليوم الانتخابات الأمة المسلمين السعودية الإسلامية
Topic #2: الله الرئيس رجب rt ان العالم اليوم الإسلام رئيس الانتخابات
Topic #3: الله الرئيس رجب rt اللهم اليوم العالم ان الإسلامي الزعيم
Topic #4: الرئيس الله rt رجب اللهم كمال الانتخابات الانتخابات_التركية العالم الرئاسية




<ipython-input-10-5e79f8f51454>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_class['processed_text'] = df_class['tweet_text'].apply(preprocess)


Top words for class 2:
Topic #0: rt ان الله الرئيس الشعب الانتخابات كمال حزب العرب nabdturkey
Topic #1: سوريا ان الله المعارضة الرئيس يقول الى انت لماذا مليون
Topic #2: ان الله اليوم الرئيس المعارضة rt الانتخابات ضد أوغلو حزب
Topic #3: الله حزب الرئيس المعارضة الشعب ضد الانتخابات رئيس العرب أوغلو
Topic #4: حزب ان الشعب أوغلو rt ضد chp الأسد الانتخابات أنه




<ipython-input-10-5e79f8f51454>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_class['processed_text'] = df_class['tweet_text'].apply(preprocess)


Top words for class 3:
Topic #0: الرئيس rt رجب عاجل اليوم رئيس قطر المعارضة الزلزال ان
Topic #1: حزب rt العدالة خلال الرئيس ان اليوم المعارضة والتنمية لأردوغان
Topic #2: rt الانتخابات الرئيس سنان الانتخابات_التركية أوغلو كمال رجب الرئاسية كليجدار
Topic #3: الرئيس rt رجب اليوم بن الله محمد trtarabi tcbestepe_ar علي
Topic #4: الرئيس rt رئيس حزب رجب الانتخابات الجمهورية أوغلو العدالة الشعب




In [ ]:
#LDA results seems meaningless

Topic Modeling Bert Topic


In [ ]:
# Install BERTopic
!pip install bertopic[all]
!pip install flair


from google.colab import files
import re
from flair.embeddings import TransformerDocumentEmbeddings
from bertopic import BERTopic
import pandas as pd

In [ ]:
# Download and load the Arabic stop words
!wget https://raw.githubusercontent.com/mohataher/arabic-stop-words/master/list.txt -O arabic_stopwords.txt
with open('arabic_stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = [line.strip() for line in file]

# Add custom stop words
custom_stopwords = [
    "ردوغان", "تركيا", "الريس", "الله", "اردوغان",
    "التركي", "لى", "طيب", "التركية", "يارب", "يا"
]
stopwords.extend(custom_stopwords)

# Function to preprocess tweets
def preprocess_tweets(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)     # Remove mentions
    text = re.sub(r'#\w+', '', text)     # Remove hashtags (optional)
    text = re.sub(r'RT[\s]+', '', text)  # Remove RT for retweets
    text = re.sub(r'[^ا-يA-Za-z\s]', '', text)  # Keep only Arabic and English letters
    text = ' '.join(word for word in text.split() if word not in stopwords)  # Remove stop words
    return text.strip()

--2024-05-12 05:56:14--  https://raw.githubusercontent.com/mohataher/arabic-stop-words/master/list.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6887 (6.7K) [text/plain]
Saving to: ‘arabic_stopwords.txt’

arabic_stopwords.tx 100%[===================>]   6.73K  --.-KB/s    in 0s      

2024-05-12 05:56:14 (47.9 MB/s) - ‘arabic_stopwords.txt’ saved [6887/6887]



In [ ]:
# Setup BERT embedding model
arabic_bert_embedding = TransformerDocumentEmbeddings('aubmindlab/bert-base-arabertv02')

# Process data and generate topics for each label
for label in [1, 2, 3]:
    df_label = df[df['cg_4_dashboard_chainofthought_few_shot'] == label]
    df_label['processed_tweets'] = df_label['tweet_text'].apply(preprocess_tweets)

    # Initialize and fit BERTopic
    topic_model = BERTopic(language="arabic", embedding_model=arabic_bert_embedding, nr_topics='auto')
    topics, probabilities = topic_model.fit_transform(df_label['processed_tweets'])

    # Create a DataFrame with topics
    df_topics = pd.DataFrame({
        "Document": df_label['tweet_text'],
        "Processed": df_label['processed_tweets'],
        "Topic": topics
    })

    # Save the DataFrame to an Excel file
    excel_filename = f"topic_assignments_label_{label}.xlsx"
    df_topics.to_excel(excel_filename, index=False)

    # Download the file
    print(f"Downloading Excel file for label {label}...")
    files.download(excel_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>